## 🎭 Personality Type (Introvert - Extrovert) Classification

Given *data about posts people have made*, let's try to predict the **personality type** of a given person.

We will use a Tensorflow RNN to make our predictions.

Data source: https://www.kaggle.com/datasets/datasnaek/mbti-type

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import tensorflow as tf

2025-05-25 12:20:38.629662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv('mbti_1.csv')
data

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


### Preprocessing

In [4]:
texts = data['posts'].copy()
labels = data['type'].copy()

In [5]:
# Process text data
data.iloc[0, 1]

"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

In [6]:
stop_words = stopwords.words('english')

texts = [text.lower() for text in texts]
texts = [text.split() for text in texts]
texts = [[word.strip() for word in text] for text in texts]
texts = [[word for word in text if word not in stop_words] for text in texts]

vocab_length = 1000

tokenizer = Tokenizer(num_words=vocab_length)
tokenizer.fit_on_texts(texts)

texts = tokenizer.texts_to_sequences(texts)

max_seq_len = np.max([len(text) for text in texts])

texts = pad_sequences(texts, maxlen=max_seq_len, padding='post')

In [7]:
texts

array([[ 87, 580, 207, ...,   0,   0,   0],
       [576, 422, 486, ...,   0,   0,   0],
       [  8,  16,  36, ...,   0,   0,   0],
       ...,
       [ 44, 408, 438, ...,   0,   0,   0],
       [511,  51, 162, ...,   0,   0,   0],
       [ 78,  76,   9, ...,   0,   0,   0]], dtype=int32)

In [8]:
data['type'].unique()

array(['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'],
      dtype=object)

In [9]:
# Process label data
label_values = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']

label_mapping = {label: int(label[0] == 'E') for label in label_values}
labels = labels.replace(label_mapping)
labels = np.array(labels)

/tmp/ipykernel_8898/2896817382.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels = labels.replace(label_mapping)


In [10]:
print("Text Sequences:\n", texts.shape)
print("\nLabels:\n", labels.shape)
print("\nMax sequence length:\n", max_seq_len)
print("\nVocab length\n", vocab_length)
print("\nLabel mapping:\n", label_mapping)

Text Sequences:
 (8675, 600)

Labels:
 (8675,)

Max sequence length:
 600

Vocab length
 1000

Label mapping:
 {'INFJ': 0, 'ENTP': 1, 'INTP': 0, 'INTJ': 0, 'ENTJ': 1, 'ENFJ': 1, 'INFP': 0, 'ENFP': 1, 'ISFP': 0, 'ISTP': 0, 'ISFJ': 0, 'ISTJ': 0, 'ESTP': 1, 'ESFP': 1, 'ESTJ': 1, 'ESFJ': 1}


In [11]:
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, train_size=0.7, random_state=123)

In [12]:
texts_train

array([[299,  31, 761, ...,   0,   0,   0],
       [317,   2, 162, ...,   0,   0,   0],
       [  2, 192, 322, ...,   0,   0,   0],
       ...,
       [139,   5,   1, ...,   0,   0,   0],
       [ 69, 111, 548, ...,   0,   0,   0],
       [404, 134, 541, ...,   0,   0,   0]], dtype=int32)

### Training

In [13]:
np.mean(labels_train)

0.23122529644268774

In [14]:
embedding_dim = 512

inputs = tf.keras.Input(shape=(max_seq_len, ))
embedding = tf.keras.layers.Embedding(
    input_dim = vocab_length,
    output_dim = embedding_dim,
    input_length = max_seq_len
)(inputs)

gru = tf.keras.layers.GRU(
        units=128,
        return_sequences = True
    )(embedding)

flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)

history = model.fit(
    texts_train,
    labels_train,
    validation_split=0.2,
    batch_size=32,
    epochs=5
)

2025-05-25 12:21:05.182913: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/5


2025-05-25 12:21:05.493158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-05-25 12:21:05.494980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-05-25 12:21:05.496315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

152/152 [==============================] - ETA: 0s - loss: 0.5148 - accuracy: 0.7797 - auc: 0.6523  

2025-05-25 12:23:55.422590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-05-25 12:23:55.424747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-05-25 12:23:55.426252: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

152/152 [==============================] - 181s 1s/step - loss: 0.5148 - accuracy: 0.7797 - auc: 0.6523 - val_loss: 0.4367 - val_accuracy: 0.8033 - val_auc: 0.8303
Epoch 2/5
152/152 [==============================] - 201s 1s/step - loss: 0.2491 - accuracy: 0.9030 - auc: 0.9505 - val_loss: 0.5088 - val_accuracy: 0.7885 - val_auc: 0.7613
Epoch 3/5
152/152 [==============================] - 190s 1s/step - loss: 0.0605 - accuracy: 0.9817 - auc: 0.9972 - val_loss: 0.6199 - val_accuracy: 0.7934 - val_auc: 0.7631
Epoch 4/5
152/152 [==============================] - 190s 1s/step - loss: 0.0102 - accuracy: 0.9994 - auc: 1.0000 - val_loss: 0.7856 - val_accuracy: 0.7934 - val_auc: 0.7535
Epoch 5/5
152/152 [==============================] - 181s 1s/step - loss: 0.0023 - accuracy: 0.9998 - auc: 1.0000 - val_loss: 0.9055 - val_accuracy: 0.7926 - val_auc: 0.7424


### Results

In [15]:
model.evaluate(texts_test, labels_test)

82/82 [==============================] - 31s 378ms/step - loss: 0.8426 - accuracy: 0.8068 - auc: 0.7703


[0.8425688743591309, 0.8067614436149597, 0.7702974677085876]